In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import dash_table
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
#df = pd.read_csv("IAT_2/IAT/Datasets/nutrition/all_nutrition.csv")#
df = pd.read_csv("/Users/srijeevsarkar/Desktop/IAT/Datasets/nutrition/all_nutrition.csv")
df["quantity"] = 1

In [3]:
df

,Restaurant,Category,Items,Cost (USD),Calories,Protein (g),Fat (g),Carbs (g),Sugar (g),Sodium (g),quantity
0,burgerking,Breakfast,"BK - Bacon, Egg & Cheese Biscuit",2.99,400,13.0,26.0,29,3,1.27,1
1,burgerking,Breakfast,BK - Breakfast Burrito Jr.,1.00,370,15.0,23.0,27,2,0.93,1
2,burgerking,Breakfast,"BK - CROISSAN’WICH® Bacon, Egg & Cheese",2.99,370,14.0,21.0,30,4,0.76,1
3,burgerking,Breakfast,BK - CROISSAN’WICH® Egg & Cheese,2.99,340,12.0,18.0,29,4,0.61,1
4,burgerking,Breakfast,"BK - CROISSAN’WICH® Ham, Egg & Cheese",2.99,370,17.0,19.0,30,5,1.03,1
...,...,...,...,...,...,...,...,...,...,...,...
387,subway,Sandwich,SW - Sweet Onion Chicken Teriyaki Sandwich,5.29,320,4.0,52.0,16,25,0.72,1
388,subway,Sandwich,SW - Turkey & Bacon Guacamole Sandwich,8.29,780,27.0,87.0,11,49,2.20,1
389,subway,Sandwich,SW - Turkey Breast Sandwich,5.29,250,3.0,41.0,6,20,0.69,1
390,subway,Sandwich,SW - Ultimate Steak Sandwich,7.49,600,35.0,42.0,6,31,1.33,1


In [ ]:
cols = ['Restaurant']
parts = [df.groupby([col, 'Category']).quantity.sum() for col in cols]
df2 = pd.concat(parts).unstack(fill_value=0)
options1 = df["Restaurant"].unique()

# Top sunburst

In [4]:
df
path = ["Restaurant", "Category", "Items"]

In [ ]:
app = dash.Dash(__name__)

In [ ]:
app.layout = html.Div(
    [
        html.H4(id="title"),
        html.Div(
            [
                dcc.Graph(
                    id="sunburst",
                        style=
                        {
                        'height': 800
                        },
                    figure=px.sunburst(df, path=path, values="Cost (USD)"),
                )
            ]
        ),
    ]
)

In [ ]:
@app.callback(
    [Output("title", "children")],
    [Input("sunburst", "clickData")],
)
def update_table(clickData):
    click_path = "ALL"
    root = False

    data = df.to_dict("records")
    if clickData:
        click_path = clickData["points"][0]["id"].split("/")
        selected = dict(zip(path, click_path))

        if "sex" in selected:
            dff = df[
                (df["Restaurant"] == selected["Restaurant"])
                & (df["Category"] == selected["Category"])
                & (df["Item"] == selected["Item"])
            ]
        elif "time" in selected:
            dff = df[(df["Restaurant"] == selected["Restaurant"]) & (df["Category"] == selected["Category"])]
        else:
            dff = df[(df["Restaurant"] == selected["Restaurant"])]
            root = True
        data = dff.to_dict("records")

        # Show all data when returning to the root from an outer leaf
        percentEntry = (clickData["points"][0]).get("percentEntry")
        if root and percentEntry == 1:
            data = df.to_dict("records")
            click_path = "ALL"

    title = f"Selected from Sunburst Chart: {' '.join(click_path)}"

    return data, title

if __name__ == "__main__":
    app.run_server(host = '127.0.0.1')

In [ ]:
df.to_dict("records")

In [ ]:
px.bar(g1[test], x="Items", y="Cost (USD)", 
                 color="Restaurant", barmode="group")